This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/limit_cycles.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import display
from pydrake.all import DirectCollocation, PiecewisePolynomial, Solve, eq
from pydrake.examples import VanDerPolOscillator

from underactuated import running_as_notebook

if running_as_notebook:
    mpld3.enable_notebook()

# Finding the limit cycle of the Van der Pol oscillator

by setting up a simple trajectory optimization, where the timestep, $h$, is a decision variables.

In [ ]:
# TODO(russt): visualize this in meshcat so we can watch the convergence again.


def vdp_limit_cycle():
    plant = VanDerPolOscillator()
    context = plant.CreateDefaultContext()

    dircol = DirectCollocation(
        plant,
        context,
        num_time_samples=61,
        minimum_timestep=0.01,
        maximum_timestep=0.5,
    )
    prog = dircol.prog()

    # Constrain all timesteps, $h[k]$, to be equal, so the trajectory breaks are evenly distributed.
    dircol.AddEqualTimeIntervalsConstraints()

    # Initial state on the surface of section (and velocity > .1).
    prog.AddBoundingBoxConstraint(
        [0.0, 0.1], [0.0, 10.0], dircol.initial_state()
    )

    # Periodicity constraint.
    prog.AddLinearConstraint(eq(dircol.final_state(), dircol.initial_state()))

    # Help the solver with an initial guess (circular trajectory).
    samples = np.linspace(0, 2 * np.pi, 10)
    x_guess = np.vstack(
        ([2 * np.sin(t) for t in samples], [2 * np.cos(t) for t in samples])
    )
    initial_x_trajectory = PiecewisePolynomial.FirstOrderHold(samples, x_guess)
    dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_trajectory)

    fig = plt.figure(figsize=(6, 6))
    (h,) = plt.plot([], [], ".-")
    plt.xlim((-2.5, 2.5))
    plt.ylim((-3.0, 3.0))
    plt.axis("equal")

    def draw_trajectory(t, x):
        h.set_xdata(x[0, :])
        h.set_ydata(x[1, :])
        fig.canvas.draw()
        if plt.get_backend() == "MacOSX":
            plt.pause(1e-10)

    if False:
        dircol.AddStateTrajectoryCallback(draw_trajectory)

    result = Solve(prog)
    assert result.is_success()

    x_trajectory = dircol.ReconstructStateTrajectory(result)

    x_knots = np.hstack(
        [
            x_trajectory.value(t)
            for t in np.linspace(
                x_trajectory.start_time(), x_trajectory.end_time(), 100
            )
        ]
    )
    plt.plot(x_knots[0, :], x_knots[1, :], "b-", marker=".")
    display(mpld3.display())


vdp_limit_cycle()